In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoImageProcessor, CvtForImageClassification, AutoConfig
import numpy as np
import optuna
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image, UnidentifiedImageError
import os
from sklearn.metrics import f1_score
from torch.utils.tensorboard import SummaryWriter
import seaborn as sns
import matplotlib.pyplot as plt

# Tentukan device untuk training (GPU jika tersedia)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters dan paths
BATCH_SIZE = 32
NUM_CLASSES = 20
IMAGE_SIZE = 224
DATASET_PATH = '/kaggle/input/d/phiard/aksara-jawa/v3/v3'

# Load dan konfigurasi model CvT-13
config = AutoConfig.from_pretrained("microsoft/cvt-13")
config.num_labels = NUM_CLASSES
image_processor = AutoImageProcessor.from_pretrained("microsoft/cvt-13")

# Custom dataset untuk memuat dataset dengan struktur folder baru
class CustomImageFolder(Dataset):
    def __init__(self, root, transform=None):
        self.samples = []
        self.targets = []
        self.transform = transform
        classes, class_to_idx = self._find_classes(root)
        self.class_to_idx = class_to_idx
        self.classes = classes
        for target_class in sorted(class_to_idx.keys()):
            class_index = class_to_idx[target_class]
            target_dir = os.path.join(root, target_class)
            if not os.path.isdir(target_dir):
                continue
            for root_, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                for fname in sorted(fnames):
                    path = os.path.join(root_, fname)
                    try:
                        with Image.open(path) as img:
                            img.verify()
                        self.samples.append((path, class_index))
                        self.targets.append(class_index)
                    except (UnidentifiedImageError, OSError):
                        print(f"Gambar korup dilewati: {path}")
                        continue

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        path, target = self.samples[index]
        with Image.open(path) as sample:
            sample = sample.convert('RGB')
        inputs = image_processor(images=sample, return_tensors="pt")
        return inputs["pixel_values"].squeeze(0), target

    def _find_classes(self, dir):
        classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx


# Fungsi untuk melatih model
def train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, epochs=10):
    best_val_acc = 0
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data).logits
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = output.max(1)
            train_correct += predicted.eq(target).sum().item()
            train_total += target.size(0)

        train_acc = 100. * train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data).logits
                loss = criterion(output, target)
                val_loss += loss.item()
                _, predicted = output.max(1)
                val_correct += predicted.eq(target).sum().item()
                val_total += target.size(0)

        val_acc = 100. * val_correct / val_total

        if val_acc > best_val_acc:
            best_val_acc = val_acc

        scheduler.step()

    return best_val_acc

# Fungsi objektif untuk Optuna
def objective(trial):
    # Tentukan hyperparameter yang dicari
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-3)
    hidden_dropout_prob = trial.suggest_uniform('hidden_dropout_prob', 0.1, 0.5)

    # Buat model dengan hyperparameter yang dicoba
    model = CvtForImageClassification.from_pretrained("microsoft/cvt-13", config=config, ignore_mismatched_sizes=True).to(device)
    model.config.hidden_dropout_prob = hidden_dropout_prob

    # Tentukan optimizers dan scheduler
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=10)

    # Tentukan loss function
    train_dataset = CustomImageFolder(os.path.join(DATASET_PATH, 'train'))
    val_dataset = CustomImageFolder(os.path.join(DATASET_PATH, 'val'))
    class_weights = compute_class_weight('balanced', classes=np.unique(train_dataset.targets), y=train_dataset.targets)
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Latih model
    best_val_acc = train_model(model, train_loader, val_loader, optimizer, criterion, scheduler)

    # Kembalikan hasil evaluasi (kita menggunakan akurasi validasi)
    return best_val_acc

# Set up Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Uji 50 kombinasi

# Hasil optimasi
print(f'Best trial: {study.best_trial.number}')
print(f'  Best learning rate: {study.best_trial.params["learning_rate"]}')
print(f'  Best weight decay: {study.best_trial.params["weight_decay"]}')
print(f'  Best dropout probability: {study.best_trial.params["hidden_dropout_prob"]}')


In [ ]:
# Tentukan device untuk training (GPU jika tersedia)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Hyperparameters dan paths
BATCH_SIZE = 32
NUM_CLASSES = 20
EPOCHS = 50
LEARNING_RATE = 0.0005
WEIGHT_DECAY = 0.00001
IMAGE_SIZE = 224
DATASET_PATH = '/kaggle/input/d/phiard/aksara-jawa/v3/v3'


In [ ]:
# Load dan konfigurasi model CvT-13
config = AutoConfig.from_pretrained("microsoft/cvt-13")
config.num_labels = NUM_CLASSES
config.hidden_dropout_prob = 0.01
image_processor = AutoImageProcessor.from_pretrained("microsoft/cvt-13")
model = CvtForImageClassification.from_pretrained(
    "microsoft/cvt-13", config=config, ignore_mismatched_sizes=True
).to(device)

In [ ]:
from torchvision import transforms

# Transformasi untuk data training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),            # Flipping horizontal acak
    transforms.RandomRotation(20),                # Rotasi acak antara -20 hingga 20 derajat
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),  # Crop acak dengan ukuran tertentu
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Variasi warna
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalisasi
])

# Transformasi untuk data validasi (tanpa augmentasi)
val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),               # Resize gambar untuk validasi
    transforms.CenterCrop(IMAGE_SIZE),           # Crop di tengah
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalisasi
])

# CustomImageFolder dengan transformasi yang telah diubah
class CustomImageFolder(Dataset):
    def __init__(self, root, transform=None):
        self.samples = []
        self.targets = []
        self.transform = transform
        classes, class_to_idx = self._find_classes(root)
        self.class_to_idx = class_to_idx
        self.classes = classes

        for target_class in sorted(class_to_idx.keys()):
            class_index = class_to_idx[target_class]
            target_dir = os.path.join(root, target_class)
            if not os.path.isdir(target_dir):
                continue
            for root_, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                for fname in sorted(fnames):
                    path = os.path.join(root_, fname)
                    try:
                        with Image.open(path) as img:
                            img.verify()
                        self.samples.append((path, class_index))
                        self.targets.append(class_index)
                    except (UnidentifiedImageError, OSError):
                        print(f"Gambar korup dilewati: {path}")
                        continue

    def __getitem__(self, index):
        path, target = self.samples[index]
        with Image.open(path) as sample:
            sample = sample.convert('RGB')
        
        # Apply transformation
        if self.transform:
            sample = self.transform(sample)
        
        return sample, target

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        path, target = self.samples[index]
        with Image.open(path) as sample:
            sample = sample.convert('RGB')
        inputs = image_processor(images=sample, return_tensors="pt")
        return inputs["pixel_values"].squeeze(0), target

    def _find_classes(self, dir):
        classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx


In [ ]:
# Load dataset dan lakukan pembagian 80/20
train_dataset = CustomImageFolder(os.path.join(DATASET_PATH, 'train'))
val_dataset = CustomImageFolder(os.path.join(DATASET_PATH, 'val'))

# Pembagian test dataset ke training dan validation 80/20
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])

# Membuat DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# Hitung class weights untuk ketidakseimbangan kelas
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_dataset.dataset.targets),
    y=train_dataset.dataset.targets
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Scheduler
scheduler = CosineAnnealingLR(optimizer, T_max=10)


In [ ]:
# Fungsi train
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        output = model(data).logits
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = output.max(1)
        correct += predicted.eq(target).sum().item()
        total += target.size(0)
    return total_loss / len(loader), 100. * correct / total


In [ ]:
# Fungsi validasi
def validate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data).logits
            loss = criterion(output, target)
            total_loss += loss.item()
            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)
            all_targets.extend(target.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
    f1 = f1_score(all_targets, all_predictions, average='weighted')
    return total_loss / len(loader), 100. * correct / total, f1, all_targets, all_predictions


In [ ]:
# Training loop
writer = SummaryWriter()
best_val_acc = 0
train_losses, train_accs, val_losses, val_accs, val_f1_scores = [], [], [], [], []

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc, val_f1, all_targets, all_predictions = validate(model, val_loader, criterion)

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    val_f1_scores.append(val_f1)

    writer.add_scalars('Loss', {'Train': train_loss, 'Val': val_loss}, epoch)
    writer.add_scalars('Accuracy', {'Train': train_acc, 'Val': val_acc}, epoch)
    writer.add_scalar('F1_Score/Val', val_f1, epoch)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_cvt_model.pth")
    
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Val F1 Score: {val_f1:.4f}")

    scheduler.step()

writer.close()


In [ ]:
# Plot confusion matrix
def plot_confusion_matrix(targets, predictions, class_names):
    cm = confusion_matrix(targets, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

plot_confusion_matrix(all_targets, all_predictions, train_dataset.dataset.classes)


In [ ]:
# Fungsi untuk menampilkan gambar per kelas dengan prediksi model
def show_predictions_per_class(model, dataset, num_classes=20, cols=5):
    model.eval()
    rows = (num_classes + cols - 1) // cols
    fig, axs = plt.subplots(rows, cols, figsize=(15, 3 * rows))
    axs = axs.flatten()

    class_images = {i: None for i in range(num_classes)}

    for i in range(len(dataset)):
        image, label = dataset[i]
        image = image.unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(image).logits
            _, predicted = output.max(1)
            if class_images[label] is None and predicted.item() == label:
                class_images[label] = (image.squeeze().cpu(), label, predicted.item())
                if all(value is not None for value in class_images.values()):
                    break

    for i in range(num_classes):
        ax = axs[i]
        if class_images[i] is not None:
            img, true_label, pred_label = class_images[i]
            ax.imshow(img.permute(1, 2, 0).numpy())
            ax.set_title(f"True: {train_dataset.dataset.classes[true_label]}, Pred: {train_dataset.dataset.classes[pred_label]}")
            ax.axis('off')
        else:
            ax.text(0.5, 0.5, 'No Prediction', ha='center', va='center')
            ax.axis('off')

    for j in range(num_classes, rows * cols):
        axs[j].axis('off')
    
    plt.tight_layout()
    plt.show()

show_predictions_per_class(model, val_dataset)


In [ ]:
# Plotting Training and Validation Loss and Accuracy
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_accs) + 1), train_accs, label='Train Accuracy')
plt.plot(range(1, len(val_accs) + 1), val_accs, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
